## EE 782: Advanced Topics in Machine Learning
### **Assignment 2:** Metric Learning and Generative AI
> **NAME:** Munish Monga

> **ROLL NO:** 22M2153

> [Github Repo Link](https://github.com/munish30monga/Metric_Learning-Generative_AI)

### Table of Contents:
- [Introduction](#introduction)
- [Importing Libraries](#importing-libraries)
- [Hyperparameters](#hyperparameters)
- [References](#references)

### Introduction <a id="introduction"></a>
**PART A: Metric Learning using Siamese Network**
> We have to train a Siamese Network on Labeled Faces in the Wild (LFW) Dataset


**PART B: Generative AI using GANs**
> We have to train a generative model for generating face images, using a GAN.

### Importing Libraries <a id="importing-libraries"></a>

In [10]:
import pathlib as pl                                            # for path handling
import glob                                                     # for path handling

### Hyperparameters <a id="hyperparameters"></a>

## **PART A:** Metric Learning using Siamese Network <a id="part-a"></a>
### Downloading Dataset
> <font color="gray">Q1. Download labeled faces in the wild (LFW) dataset</font>

LFW Dataset was downloaded and extracted into the following path: `./dataset/lfw/`

### Exploring Dataset <a id="exploring-dataset"></a>
> <font color="gray">Q2. Get the number of persons who have more than one image</font>

In [12]:
# data_dir = pl.Path('./dataset/lfw/')     # Setting path to the data directory
# persons_dirs = data_dir.glob("*")  # Use glob to get all subdirectories in the provided path
# persons_dict = {
#     person_dir.name: len(list(persons_dirs.glob("*.jpg")))
#     for person_dir in persons_dirs
# }
# person_names = list(persons_dict.keys())
# print(f"Number of persons: {len(person_names)}")
# print(f"Number of images: {sum(persons_dict.values())}")
# print(f"First 10 persons: {person_names[:10]}")